In [1]:
import pandas as pd
def load_csv_from_github(file_name):
 base_url = "https://raw.githubusercontent.com/marymorkos/SalesPlaybookDS5640/refs/heads/main/"
 return pd.read_csv(base_url + file_name)

In [2]:
companies_df = load_csv_from_github("anonymized_hubspot_companies.csv")
companies_df.head()

C:\Users\laura\AppData\Local\Temp\ipykernel_34804\2943978199.py:4: DtypeWarning: Columns (24,25,28,32,45) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(base_url + file_name)


,Parent Company,CCaaS,Annual Revenue,Target Account,Associated Contact,Number of Form Submissions,Total Agents,Web Technologies,Close Date,# of Agents Total,...,BPO Program,Primary Sub-Industry,Number of Sessions,WFM,Country/Region,Industry,Create Date,Company name,Last Modified Date,BPO Program Tier
0,NaN,NaN,1.000000e+09,NaN,Contact_ef780380,0.0,NaN,Route 53,NaN,NaN,...,NaN,NaN,0.0,NaN,United States,HVAC and plumbing supply,2024-10-30 10:51,Company_4fc73a2a,2025-02-14 13:39,NaN
1,NaN,NaN,NaN,NaN,Contact_93373ba5,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Pest Control,2024-09-12 18:30,Company_f3f7e884,2025-02-14 13:39,NaN
2,NaN,NaN,5.000000e+07,NaN,Contact_2e8e0993,0.0,NaN,Youtube; App Nexus; Google Tag Manager; Facebo...,NaN,NaN,...,NaN,NaN,0.0,NaN,United States,Pest Control,2024-09-03 10:44,Company_89929bed,2025-02-14 13:39,NaN
3,NaN,Five9,5.000000e+07,NaN,Contact_635e44ed,0.0,NaN,Postmark; Facebook Advertiser; Salesforce; Goo...,NaN,NaN,...,NaN,NaN,0.0,NaN,United States,Pest Control,2024-08-02 11:36,Company_33dbf591,2025-02-14 13:39,NaN
4,NaN,Five9,1.000000e+08,NaN,Contact_8d055096,0.0,NaN,Microsoft Office 365; Google Tag Manager; Reca...,NaN,NaN,...,NaN,NaN,0.0,NaN,United States,Pest Control,2024-08-02 11:36,Company_a3079821,2025-02-14 13:39,NaN


In [4]:
companies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19851 entries, 0 to 19850
Data columns (total 46 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Parent Company                31 non-null     object 
 1   CCaaS                         2086 non-null   object 
 2   Annual Revenue                18341 non-null  float64
 3   Target Account                33 non-null     object 
 4   Associated Contact            17792 non-null  object 
 5   Number of Form Submissions    17248 non-null  float64
 6   Total Agents                  0 non-null      float64
 7   Web Technologies              18607 non-null  object 
 8   Close Date                    71 non-null     object 
 9   # of Agents Total             42 non-null     float64
 10  Number of times contacted     11544 non-null  float64
 11  Contact with Primary Company  17248 non-null  object 
 12  ICP Fit Level                 5983 non-null   object 
 13  R